In [ ]:
import datetime
import time
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from netCDF4 import Dataset, num2date, date2num
import numpy as np
from scipy.ndimage import gaussian_filter
from shapely.geometry.polygon import LinearRing
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import xlrd

In [ ]:
os.environ["CARTOPY_USER_BACKGROUNDS"] = '/Users/cjmasiel/Desktop/Cartopy/BG/'

In [ ]:
def create_map_background():
    dataproj = ccrs.PlateCarree()
    fig=plt.figure(figsize=(25, 25))
    ax=plt.subplot(111, projection=dataproj)
    ax.set_extent([-10, -110, 5, 50],ccrs.PlateCarree())
    ax.coastlines('50m', linewidth=1.5)
    ax.add_feature(cfeature.STATES, linewidth=1.0)
    ax.add_feature(cfeature.BORDERS, linewidth=1.0)
    gl = ax.gridlines(color='gray',alpha=0.5,draw_labels=True)
    gl.xlabels_top, gl.ylabels_right = False, False
    gl.xlabel_style, gl.ylabel_style = {'fontsize': 16}, {'fontsize': 16}
    gl.xlocator = mticker.FixedLocator([0,-10, -20,-30, -40,-50, -60,-70, -80,-90,-100,-110])
    gl.ylocator = mticker.FixedLocator([0, 10, 20, 30, 40, 50])
    gl.xformatter = LongitudeFormatter(zero_direction_label=True)
    gl.yformatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(gl.xformatter)
    ax.yaxis.set_major_formatter(gl.yformatter)
    ###############################################################################################
    ax.background_img(name='BM', resolution='high')
    ###############################################################################################
#     plt.scatter(-180,-40,alpha = 1.0, color = '#5ebaff', label = 'Tropical Depression')
#     plt.scatter(-180,-40,alpha = 1.0, color='#00faf4', label = 'Tropical Storm')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffffcc', label = 'Category 1')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffe775', label = 'Category 2')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffc140', label = 'Category 3')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ff8f20', label = 'Category 4')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ff6060', label = 'Category 5')
#     plt.scatter(-180,-40,alpha = 1.0, color='#5ebaff', marker  = 'v', label = 'Extra-Tropical')
#     plt.scatter(-180,-40,alpha = 1.0, color='#00faf4', marker = 'v', label = 'Sub-Tropical Storm')
    plt.legend(loc = 'upper left')
    return fig, ax

In [ ]:
#Next, I want to look at the location of hurricanes and all Tropical Storms

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the base color as '#00faf4'
base_color = 'red'

# Define the number of steps in the color scale
num_steps = 20

# Create a linearly spaced array of alpha values from 0 to 1\n",
alphas = np.linspace(0, 1, num_steps)

# Create a list of colors with varying alpha values\n",
colors = [mcolors.to_rgba(base_color, alpha=a) for a in alphas]

# Create a color map using the list of colors
cmap_new = mcolors.ListedColormap(colors)

def RIdinsity(x,y,z):
    plt.title('Center Location of Tropical Cyclones in the NATL (1980-2022)', {"fontsize": 16}, loc = 'left')
    plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES ={z}', {"fontsize": 12}, loc = 'Right')
    plot = plt.hist2d(x, y, bins=[list(range(-110, -10)), list(range(5, 50))], cmap=cmap_new)
    plt.clim(0,20)
    cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
    cbar.ax.set_xlabel('1 x 1 Degree Density', fontsize = 20)

In [ ]:
#Next, I want to average intensity change
#First, we have to specify a range of years in which we consider

intensity_change = []
lat = []
lon = []

#This is the list of unique storms I want to consider\n
CODE = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            year = float(values[0][4:8])
            for i in range(1980,2023):
                if year == i:
                    code = values[0]
                    CODE.append(code)
        else:
            lines_to_read -=1 

In [ ]:
#Now that we have the storms we are interested in looking, we will loop through the same list as before\n",
#However, this time we will have a bunch of logical \n",

VMAX  = []
LAT_O = []
LON_O = []
CODE_O = []
TIME  = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            value_lines_to_read = int(values[2]) 
            code = str(values[0])
        else:
            latitude  = float(values[4][:-1])
            longitude = -1 * float(values[5][:-1])
            vmax      = float(values[6])
            nature    = str(values[3])
            time      = str(values[1])
            for i in CODE:
                if code == i:        #Just looking to see if the AL number is equal to the one in our list\n",
                    if time not in (' 0000', ' 0600', ' 1200', ' 1800'):
                        #print("Do not Consider, " "Storm =", code, "time =", time)
                        continue
                    elif nature in (' TD', ' TS', ' HU'):
                    #elif nature in (' HU'):
                        VMAX.append(vmax)
                        LAT_O.append(latitude)
                        LON_O.append(longitude)
                        CODE_O.append(code)       
            #now we continue parsing through our data set\n",
            lines_to_read -=1 


print("Length of number of Storms appended = ", len(CODE_O))
print("Length of intensity list = ", len(VMAX))

In [ ]:
#now, lets plot the whole 1980-2022 Atlantic Distribution

fig, ax = create_map_background()

RIdinsity(LON_O,LAT_O,len(LON_O))

In [ ]:
#Now, let us split this up into two groups, 1980-2001, 2002-2022
#Next, I want to average intensity change
#First, we have to specify a range of years in which we consider
#This is the list of unique storms I want to consider\n
CODE_1980_2000 = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            year = float(values[0][4:8])
            for i in range(1980,2001):
                if year == i:
                    code = values[0]
                    CODE_1980_2000.append(code)
        else:
            lines_to_read -=1 

In [ ]:
#Now that we have the storms we are interested in looking, we will loop through the same list as before\n",
#However, this time we will have a bunch of logical \n",

VMAX_80_00  = []
LAT_80_00 = []
LON_80_00 = []
CODE_80_00 = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            value_lines_to_read = int(values[2]) 
            code = str(values[0])
        else:
            latitude  = float(values[4][:-1])
            longitude = -1 * float(values[5][:-1])
            vmax      = float(values[6])
            nature    = str(values[3])
            time      = str(values[1])
            for i in CODE_1980_2000:
                if code == i:        #Just looking to see if the AL number is equal to the one in our list\n",
                    if time not in (' 0000', ' 0600', ' 1200', ' 1800'):
                        #print("Do not Consider, " "Storm =", code, "time =", time)
                        continue
                    elif nature in (' TD', ' TS', ' HU'):
                    #elif nature in (' HU'):
                        VMAX_80_00.append(vmax)
                        LAT_80_00.append(latitude)
                        LON_80_00.append(longitude)
                        CODE_80_00.append(code)       
            #now we continue parsing through our data set\n",
            lines_to_read -=1 


print("Length of number of Storms appended = ", len(CODE_80_00))
print("Length of intensity list = ", len(VMAX_80_00))

In [ ]:
#Now, let us split this up into two groups, 1980-2001, 2002-2022
#Next, I want to average intensity change
#First, we have to specify a range of years in which we consider
#This is the list of unique storms I want to consider\n
CODE_2001_2022 = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            year = float(values[0][4:8])
            for i in range(2001,2023):
                if year == i:
                    code = values[0]
                    CODE_2001_2022.append(code)
        else:
            lines_to_read -=1 

In [ ]:
print(len(CODE_2001_2022))

In [ ]:
#Now that we have the storms we are interested in looking, we will loop through the same list as before\n",
#However, this time we will have a bunch of logical \n",

VMAX_01_22  = []
LAT_01_22 = []
LON_01_22 = []
CODE_01_22 = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            value_lines_to_read = int(values[2]) 
            code = str(values[0])
        else:
            latitude  = float(values[4][:-1])
            longitude = -1 * float(values[5][:-1])
            vmax      = float(values[6])
            nature    = str(values[3])
            time      = str(values[1])
            for i in CODE_2001_2022:
                if code == i:        #Just looking to see if the AL number is equal to the one in our list\n",
                    if time not in (' 0000', ' 0600', ' 1200', ' 1800'):
                        #print("Do not Consider, " "Storm =", code, "time =", time)
                        continue
                    elif nature in (' TD', ' TS', ' HU'):
                    #elif nature in (' HU'):
                        VMAX_01_22.append(vmax)
                        LAT_01_22.append(latitude)
                        LON_01_22.append(longitude)
                        CODE_01_22.append(code)       
            #now we continue parsing through our data set\n",
            lines_to_read -=1 


print("Length of number of Storms appended = ", len(CODE_01_22))
print("Length of intensity list = ", len(VMAX_01_22))

In [ ]:
#Next, I want to look at the location of hurricanes and all Tropical Storms

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the base color as '#00faf4'
base_color = 'red'

# Define the number of steps in the color scale
num_steps = 10

# Create a linearly spaced array of alpha values from 0 to 1\n",
alphas = np.linspace(0, 1, num_steps)

# Create a list of colors with varying alpha values\n",
colors = [mcolors.to_rgba(base_color, alpha=a) for a in alphas]

# Create a color map using the list of colors
cmap_new = mcolors.ListedColormap(colors)

def RIdinsity(x,y,z):
    plt.title('Center Location of Tropical Cyclones in the NATL (1980-2000)', {"fontsize": 16}, loc = 'left')
    plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES ={z}', {"fontsize": 12}, loc = 'Right')
    plot = plt.hist2d(x, y, bins=[list(range(-110, -10)), list(range(5, 50))], cmap=cmap_new)
    plt.clim(0,10)
    cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
    cbar.ax.set_xlabel('1 x 1 Degree Density', fontsize = 20)

In [ ]:
#now, lets plot the whole 1980-2022 Atlantic Distribution

fig, ax = create_map_background()

RIdinsity(LON_80_00,LAT_80_00,len(LON_80_00))

In [ ]:
#Next, I want to look at the location of hurricanes and all Tropical Storms

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the base color as '#00faf4'
base_color = 'red'

# Define the number of steps in the color scale
num_steps = 10

# Create a linearly spaced array of alpha values from 0 to 1\n",
alphas = np.linspace(0, 1, num_steps)

# Create a list of colors with varying alpha values\n",
colors = [mcolors.to_rgba(base_color, alpha=a) for a in alphas]

# Create a color map using the list of colors
cmap_new = mcolors.ListedColormap(colors)

def RIdinsity(x,y,z):
    plt.title('Center Location of Tropical Cyclones in the NATL (2001-2023)', {"fontsize": 16}, loc = 'left')
    plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES ={z}', {"fontsize": 12}, loc = 'Right')
    plot = plt.hist2d(x, y, bins=[list(range(-110, -10)), list(range(5, 50))], cmap=cmap_new)
    plt.clim(0,10)
    cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
    cbar.ax.set_xlabel('1 x 1 Degree Density', fontsize = 20)

In [ ]:
#now, lets plot the whole 1980-2022 Atlantic Distribution

fig, ax = create_map_background()

RIdinsity(LON_01_22,LAT_01_22,len(LON_01_22))

In [ ]:
#now, we want to take the difference field of these two plots to see where more or less storms are occuring

#xedges = list(range(-110, -10))
#yedges = list(range(0, 50))

fig, ax = create_map_background()

#Assuming lon_6, lat_6, and intensity_change_6 are your data arrays
xedges = list(range(-110, -10))
yedges = list(range(5, 50))

hist_1, xedges, yedges = np.histogram2d(LON_80_00, LAT_80_00, bins=[xedges, yedges])
hist_2, _, _           = np.histogram2d(LON_01_22, LAT_01_22, bins=[xedges, yedges])

#calculate the difference between the two histrograms:
hist_diff = hist_2 - hist_1

#Tranpose our data to get it into the form that is useful
hist_diff_n = hist_diff.T

hist_diff_n[hist_diff_n == 0] = np.nan

#Create the plot
plt.pcolormesh(xedges, yedges, hist_diff_n, cmap='seismic', vmax=10, vmin=-10)
cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
cbar.ax.set_xlabel('Difference in 1° x 1° track density)', fontsize = 20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.title('Difference in Density Field of NATL Tropical Cyclones (2000 to 2022 minus 1980 to 2000)', {"fontsize": 20}, loc = 'left')
plt.title(f'Product By: Cameron Masiello', {"fontsize": 20}, loc = 'Right')

In [ ]:
#Now, let us make 6 landfall comparisons between the two timeframes
#Now, we shall make a plot looking at the intensity at landfall
#Now that we have the storms we are interested in looking, we will loop through the same list as before
#However, this time we will have a bunch of logical 

VMAX_80_00_Land  = []
LAT_80_00_Land = []
LON_80_00_Land = []
CODE_80_00_Land = []
TIME_80_00_Land  = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            value_lines_to_read = int(values[2]) 
            code = str(values[0])
        else:
            latitude  = float(values[4][:-1])
            longitude = -1 * float(values[5][:-1])
            vmax      = float(values[6])
            nature    = str(values[3])
            time      = str(values[1])
            indicator = str(values[2])
            #print(indicator)
            for i in CODE_1980_2000:
                if code == i:        #Just looking to see if the AL number is equal to the one in our list
                    if indicator not in (' L'):
                        #print("Do not Consider, " "Storm =", code, "time =", time)
                        continue
                    elif nature in (' TD', ' TS', ' HU'):
                        VMAX_80_00_Land.append(vmax),
                        LAT_80_00_Land.append(latitude)
                        LON_80_00_Land.append(longitude)
                        CODE_80_00_Land.append(code)     
            #now we continue parsing through our data set\n",
            lines_to_read -=1 

print("Length of number of Storms appended =", len(CODE_80_00_Land))
print("Length of intensity list =", len(VMAX_80_00_Land))

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

#Here we are making a new color scale for landfalling information
def Color_Scale_Landfall():
    newcmp = LinearSegmentedColormap.from_list("", [
    (0/137, '#5ebaff'),
    (34/137, '#00faf4'),
    (64/137, '#ffffcc'),
    (83/137, '#ffe775'),
    (96/137, '#ffc140'),
    (113/137, '#ff8f20'),
    (137/137, '#ff6060')])

    return newcmp

In [ ]:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming lon_6, lat_6, and intensity_change_6 are your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_80_00_Land, LAT_80_00_Land, bins=[xedges, yedges])
sums, _, _ = np.histogram2d(LON_80_00_Land, LAT_80_00_Land, weights=VMAX_80_00_Land, bins=[xedges, yedges])

# Calculate the ratio of sums to counts, handling potential division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    ratio = np.where(counts > 0, sums / counts, np.nan)

#Tranpose our data to get it into the form that is useful
ratio_n_80_00_Land = ratio.T

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, ratio_n_80_00_Land, cmap = Color_Scale_Landfall(), vmin= 0 , vmax= 137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Average Landfall Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Average Landfall Intensity (knots) of NATL Tropical Cyclones (1980-2000)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_80_00_Land)}', fontsize=20, loc='right')

In [ ]:
#Now, let us make 6 landfall comparisons between the two timeframes
#Now, we shall make a plot looking at the intensity at landfall
#Now that we have the storms we are interested in looking, we will loop through the same list as before
#However, this time we will have a bunch of logical 

VMAX_01_22_Land  = []
LAT_01_22_Land = []
LON_01_22_Land = []
CODE_01_22_Land = []
TIME_01_22_Land  = []

with open('/Users/cjmasiel/desktop/HURDAT2.txt', 'r') as ds:
    lines_to_read = 0
    for line in ds:
        values = line.split(",")
        if lines_to_read == 0:
            lines_to_read = int(values[2])
            value_lines_to_read = int(values[2]) 
            code = str(values[0])
        else:
            latitude  = float(values[4][:-1])
            longitude = -1 * float(values[5][:-1])
            vmax      = float(values[6])
            nature    = str(values[3])
            time      = str(values[1])
            indicator = str(values[2])
            #print(indicator)
            for i in CODE_2001_2022:
                if code == i:        #Just looking to see if the AL number is equal to the one in our list
                    if indicator not in (' L'):
                        #print("Do not Consider, " "Storm =", code, "time =", time)
                        continue
                    elif nature in (' TD', ' TS', ' HU'):
                        VMAX_01_22_Land.append(vmax),
                        LAT_01_22_Land.append(latitude)
                        LON_01_22_Land.append(longitude)
                        CODE_01_22_Land.append(code)     
            #now we continue parsing through our data set\n",
            lines_to_read -=1 

print("Length of number of Storms appended =", len(CODE_01_22_Land))
print("Length of intensity list =", len(VMAX_01_22_Land))

In [ ]:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming lon_6, lat_6, and intensity_change_6 are your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_01_22_Land, LAT_01_22_Land, bins=[xedges, yedges])
sums, _, _ = np.histogram2d(LON_01_22_Land, LAT_01_22_Land, weights=VMAX_01_22_Land, bins=[xedges, yedges])

# Calculate the ratio of sums to counts, handling potential division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    ratio = np.where(counts > 0, sums / counts, np.nan)

#Tranpose our data to get it into the form that is useful
ratio_n_01_22_Land = ratio.T

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, ratio_n_01_22_Land, cmap = Color_Scale_Landfall(), vmin= 0 , vmax= 137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Average Landfall Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Average Landfall Intensity (knots) of NATL Tropical Cyclones (2001-2022)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_01_22_Land)}', fontsize=20, loc='right')

In [ ]:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming you have lon_6, lat_6, and intensity_change_6 as your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_80_00_Land, LAT_80_00_Land, bins=[xedges, yedges])

# Create an empty array to store the maximum intensity for each grid cell
max_intensity = np.full((len(yedges) - 1, len(xedges) - 1), np.nan)

for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        lon_mask = (LON_80_00_Land >= xedges[i]) & (LON_80_00_Land < xedges[i + 1])
        lat_mask = (LAT_80_00_Land >= yedges[j]) & (LAT_80_00_Land < yedges[j + 1])
        cell_mask = lon_mask & lat_mask

        if np.any(cell_mask):
            max_intensity[j, i] = np.max(np.array(VMAX_80_00_Land)[cell_mask])

# Tranpose our data to get it into the form that is useful
max_intensity_n_80_00_Land = max_intensity

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, max_intensity_n_80_00_Land, cmap=Color_Scale_Landfall(), vmin=0, vmax=137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Maximum Landfall Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Maximum Landfall Intensity (knots) of NATL Tropical Cyclones (1980-2000)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_80_00_Land)}', fontsize=20, loc='right')

In [ ]:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming you have lon_6, lat_6, and intensity_change_6 as your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_01_22_Land, LAT_01_22_Land, bins=[xedges, yedges])

# Create an empty array to store the maximum intensity for each grid cell
max_intensity = np.full((len(yedges) - 1, len(xedges) - 1), np.nan)

# Convert your lists to NumPy arrays
LON_01_22_Land = np.array(LON_01_22_Land)
LAT_01_22_Land = np.array(LAT_01_22_Land)

for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        lon_mask = (LON_01_22_Land >= xedges[i]) & (LON_01_22_Land < xedges[i + 1])
        lat_mask = (LAT_01_22_Land >= yedges[j]) & (LAT_01_22_Land < yedges[j + 1])
        cell_mask = lon_mask & lat_mask

        if np.any(cell_mask):
            max_intensity[j, i] = np.max(np.array(VMAX_01_22_Land)[cell_mask])

# Tranpose our data to get it into the form that is useful
max_intensity_n_01_22_Land = max_intensity

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, max_intensity_n_01_22_Land, cmap=Color_Scale_Landfall(), vmin=0, vmax=137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Maximum Landfall Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Maximum Landfall Intensity (knots) of NATL Tropical Cyclones (2001-2022)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_01_22_Land)}', fontsize=20, loc='right')

In [ ]:
#Now that we have our 4 lists -Name, Intensity, Lat, Long- of equal lengths for, we will no perform a while loop\n",
#for 0,6,12,18 UTC value times, we will now run calculations for intensity change\n",

# VMAX_80_00  = []
# LAT_80_00 = []
# LON_80_00 = []
# CODE_80_00 = []

intensity_change_80_00_6 = []
lat_80_00_6 = []
lon_80_00_6 = []

n = 0
while (n + 1) < len(VMAX_80_00):
    if CODE_80_00[n+1] == CODE_80_00[n]:  #We are only going to do calculations on the data if 
        intensity_change_80_00_6 += [VMAX_80_00[n+1]-VMAX_80_00[n]]
        lat_80_00_6 += [LAT_80_00[n+1]]
        lon_80_00_6 += [LON_80_00[n+1]]
    n += 1

print(len(intensity_change_80_00_6))
print(len(lat_80_00_6))
print(len(lon_80_00_6))

In [ ]:
intensity_change_01_22_6 = []
lat_01_22_6 = []
lon_01_22_6 = []

n = 0
while (n + 1) < len(VMAX_01_22):
    if CODE_01_22[n+1] == CODE_01_22[n]:  #We are only going to do calculations on the data if 
        intensity_change_01_22_6 += [VMAX_01_22[n+1]-VMAX_01_22[n]]
        lat_01_22_6 += [LAT_01_22[n+1]]
        lon_01_22_6 += [LON_01_22[n+1]]
    n += 1

print(len(intensity_change_01_22_6))
print(len(lat_01_22_6))
print(len(lon_01_22_6))

In [ ]:
#Now that we have automated Intensity change by years and have put in the necessary fail measure to verify
#we are only looking at the 6 hour intensity change for individual storms + tropical disturbances
#next, we will look at the average intensity change per 1x1 degree lat/long

fig, ax = create_map_background()

#Assuming lon_6, lat_6, and intensity_change_6 are your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(lon_80_00_6, lat_80_00_6, bins=[xedges, yedges])
sums, _, _ = np.histogram2d(lon_80_00_6, lat_80_00_6, weights=intensity_change_80_00_6, bins=[xedges, yedges])

# Calculate the ratio of sums to counts, handling potential division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    ratio = np.where(counts > 0, sums / counts, np.nan)

#Tranpose our data to get it into the form that is useful
ratio_n_80_00 = ratio.T

# Create the plot
plt.pcolormesh(xedges, yedges, ratio_n_80_00, cmap='seismic', vmax=20, vmin=-20)
cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
cbar.ax.set_xlabel('6 hour intensity change (kt)', fontsize = 20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.title('Average 6 hour Intensity Change of NATL Tropical Cyclones (1980-2000)', {"fontsize": 20}, loc = 'left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(lat_80_00_6)}', {"fontsize": 20}, loc = 'Right')

In [ ]:
#Now that we have automated Intensity change by years and have put in the necessary fail measure to verify
#we are only looking at the 6 hour intensity change for individual storms + tropical disturbances
#next, we will look at the average intensity change per 1x1 degree lat/long

fig, ax = create_map_background()

#Assuming lon_6, lat_6, and intensity_change_6 are your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(lon_01_22_6, lat_01_22_6, bins=[xedges, yedges])
sums, _, _ = np.histogram2d(lon_01_22_6, lat_01_22_6, weights=intensity_change_01_22_6, bins=[xedges, yedges])

# Calculate the ratio of sums to counts, handling potential division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    ratio = np.where(counts > 0, sums / counts, np.nan)

#Tranpose our data to get it into the form that is useful
ratio_n_01_22 = ratio.T

# Create the plot
plt.pcolormesh(xedges, yedges, ratio_n_01_22, cmap='seismic', vmax=20, vmin=-20)
cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
cbar.ax.set_xlabel('6 hour intensity change (kt)', fontsize = 20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.title('Average 6 hour Intensity Change of NATL Tropical Cyclones (2001-2022)', {"fontsize": 20}, loc = 'left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(lat_01_22_6)}', {"fontsize": 20}, loc = 'Right')

In [ ]:
#Now, let us calcualte the maxmimum intensity of grid cells in the two time frames:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming you have lon_6, lat_6, and intensity_change_6 as your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_80_00, LAT_80_00, bins=[xedges, yedges])

# Create an empty array to store the maximum intensity for each grid cell
max_intensity = np.full((len(yedges) - 1, len(xedges) - 1), np.nan)

# Convert your lists to NumPy arrays
LON_80_00 = np.array(LON_80_00)
LAT_80_00 = np.array(LAT_80_00)

for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        lon_mask = (LON_80_00 >= xedges[i]) & (LON_80_00 < xedges[i + 1])
        lat_mask = (LAT_80_00 >= yedges[j]) & (LAT_80_00 < yedges[j + 1])
        cell_mask = lon_mask & lat_mask

        if np.any(cell_mask):
            max_intensity[j, i] = np.max(np.array(VMAX_80_00)[cell_mask])

# Tranpose our data to get it into the form that is useful
max_intensity_n_80_00 = max_intensity

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, max_intensity_n_80_00, cmap=Color_Scale_Landfall(), vmin=0, vmax=137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Maximum Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Maximum Intensity per 1° x 1° cell of NATL Tropical Cyclones (1980-2000)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_80_00)}', fontsize=20, loc='right')

In [ ]:
#Now, let us calcualte the maxmimum intensity of grid cells in the two time frames:
#this cell will plot the average intensity at landfall from 1980-2022
fig, ax = create_map_background()

# Assuming you have lon_6, lat_6, and intensity_change_6 as your data arrays
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

counts, _, _ = np.histogram2d(LON_01_22, LAT_01_22, bins=[xedges, yedges])

# Create an empty array to store the maximum intensity for each grid cell
max_intensity = np.full((len(yedges) - 1, len(xedges) - 1), np.nan)

# Convert your lists to NumPy arrays
LON_01_22 = np.array(LON_01_22)
LAT_01_22 = np.array(LAT_01_22)

for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        lon_mask = (LON_01_22 >= xedges[i]) & (LON_01_22 < xedges[i + 1])
        lat_mask = (LAT_01_22 >= yedges[j]) & (LAT_01_22 < yedges[j + 1])
        cell_mask = lon_mask & lat_mask

        if np.any(cell_mask):
            max_intensity[j, i] = np.max(np.array(VMAX_01_22)[cell_mask])

# Tranpose our data to get it into the form that is useful
max_intensity_n_01_22 = max_intensity

# Create the plot using the custom colormap and bounds
plt.pcolormesh(xedges, yedges, max_intensity_n_01_22, cmap=Color_Scale_Landfall(), vmin=0, vmax=137)

# Create a color bar with custom boundaries and labels
cbar = plt.colorbar(orientation='horizontal', pad=0.05, shrink=1, aspect=50, extend='both')

# Set the tick locations
cbar.set_ticks([34, 64, 96, 113, 137])

# Set the tick labels
cbar.set_ticklabels([34, 64, 96, 113, 137])

cbar.ax.set_xlabel('Maximum Intensity (knots)', fontsize=20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Maximum Intensity per 1° x 1° cell of NATL Tropical Cyclones (2001-2022)', fontsize=20, loc='left')
plt.title(f'Product By: Cameron Masiello \n NUMBER OF CASES = {len(LAT_01_22)}', fontsize=20, loc='right')

In [ ]:
#now, let us make a difference between the present and past with a specific interest of points over land:

fig, ax = create_map_background()

#bin edges
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

#calculate the difference between the two histrograms:
hist_diff_int = max_intensity_n_01_22 - max_intensity_n_80_00

#Create the plot
plt.pcolormesh(xedges, yedges, hist_diff_int, cmap='seismic', vmax=20, vmin=-20)
cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
cbar.ax.set_xlabel('Difference in 1° x 1° Maximum Intensity (Knots))', fontsize = 20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.title('Difference in 1° x 1° Maximum Intensity of NATL Tropical Cyclones (2001-2022) minus (1980-2000)', {"fontsize": 20}, loc = 'left')
plt.title(f'Product By: Cameron Masiello', {"fontsize": 20}, loc = 'Right')

In [ ]:
#now, let us just zoom in on the Eastern Coast of the US

def create_map_background():
    dataproj = ccrs.PlateCarree()
    fig=plt.figure(figsize=(25, 25))
    ax=plt.subplot(111, projection=dataproj)
    ax.set_extent([-65, -110, 20, 50],ccrs.PlateCarree())
    ax.coastlines('50m', linewidth=1.5)
    ax.add_feature(cfeature.STATES, linewidth=1.0)
    ax.add_feature(cfeature.BORDERS, linewidth=1.0)
    gl = ax.gridlines(color='gray',alpha=0.5,draw_labels=True)
    gl.xlabels_top, gl.ylabels_right = False, False
    gl.xlabel_style, gl.ylabel_style = {'fontsize': 16}, {'fontsize': 16}
    gl.xlocator = mticker.FixedLocator([0,-10, -20,-30, -40,-50, -60,-70, -80,-90,-100,-110])
    gl.ylocator = mticker.FixedLocator([0, 10, 20, 30, 40, 50])
    gl.xformatter = LongitudeFormatter(zero_direction_label=True)
    gl.yformatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(gl.xformatter)
    ax.yaxis.set_major_formatter(gl.yformatter)
    ###############################################################################################
    ax.background_img(name='BM', resolution='high')
    ###############################################################################################
#     plt.scatter(-180,-40,alpha = 1.0, color = '#5ebaff', label = 'Tropical Depression')
#     plt.scatter(-180,-40,alpha = 1.0, color='#00faf4', label = 'Tropical Storm')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffffcc', label = 'Category 1')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffe775', label = 'Category 2')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ffc140', label = 'Category 3')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ff8f20', label = 'Category 4')
#     plt.scatter(-180,-40,alpha = 1.0, color='#ff6060', label = 'Category 5')
#     plt.scatter(-180,-40,alpha = 1.0, color='#5ebaff', marker  = 'v', label = 'Extra-Tropical')
#     plt.scatter(-180,-40,alpha = 1.0, color='#00faf4', marker = 'v', label = 'Sub-Tropical Storm')
    plt.legend(loc = 'upper left')
    return fig, ax

In [ ]:
#now, let us make a difference between the present and past with a specific interest of points over land:

fig, ax = create_map_background()

#bin edges
xedges = list(range(-110, -10))
yedges = list(range(0, 50))

#calculate the difference between the two histrograms:
hist_diff_int = max_intensity_n_01_22 - max_intensity_n_80_00

#Create the plot
plt.pcolormesh(xedges, yedges, hist_diff_int, cmap='seismic', vmax=20, vmin=-20)
cbar = plt.colorbar(orientation = 'horizontal', pad = .05, shrink = 1, aspect = 50, extend = 'both')
cbar.ax.set_xlabel('Difference in 1° x 1° Maximum Intensity (Knots))', fontsize = 20)

# Add labels and title
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.title('Difference in 1° x 1° Maximum Intensity of NATL Tropical Cyclones (2001-2022) minus (1980-2000)', {"fontsize": 20}, loc = 'left')
plt.title(f'Product By: Cameron Masiello', {"fontsize": 20}, loc = 'Right')